# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# импорт библиотеки pandas
import pandas as pd
# импортируем pymystem3
from pymystem3 import Mystem
# импорт счётчика для лематизатора слов
from collections import Counter

In [2]:
# чтение файла с данными из папки /datasets и сохранение в df
df = pd.read_csv('data.csv')

In [3]:
# получение первых 10 строк таблицы df
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# перечень названий столбцов таблицы df
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [5]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [6]:
# просмотр значений по столбцам для поиска проблем, аномалий и общей информации о значениях в таблице
print(*[df[x].value_counts() for x in df])

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64 -986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64 35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64 среднее   

In [7]:
# проверяю отдельно значения стажа отрицательные и положительные 
# исходя из предположения по шапке таблицы о разных значениях
print('Отрицательные значения стажа')
print(df[df['days_employed'] < 0]['days_employed'].sort_values(ascending=False))
print('Положительные значения стажа')
print(df[df['days_employed'] > 0][ 'days_employed'].sort_values(ascending=False))

Отрицательные значения стажа
17437      -24.141633
8336       -24.240695
6157       -30.195337
9683       -33.520665
2127       -34.701045
             ...     
16825   -16119.687737
17838   -16264.699501
7329    -16593.472817
4299    -17615.563266
16335   -18388.949901
Name: days_employed, Length: 15906, dtype: float64
Положительные значения стажа
6954     401755.400475
10006    401715.811749
7664     401675.093434
2156     401674.466633
7794     401663.850046
             ...      
7229     328827.345667
14783    328795.726728
17782    328771.341387
9328     328734.923996
20444    328728.720605
Name: days_employed, Length: 3445, dtype: float64


**Вывод**

Итак, в таблице 12 столбцов. 

Столбцы:
         `children`, `dob_years`, `education_id`, `family_status_id`, `debt` имеют тип int64;         
         `days_employed`, `total_income` имеют тип float64;         
         `education`, `family_status`, `gender`, `income_type`, `purpose` имеют тип object

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

Как видим, стилистически и семантически название колонок верно.

Количество значений во всех колонках одинаково, кроме столбцов `days_employed`, `total_income`, в которых количество значений одинково. 

В данных, в этих колонках содержатся пропущенные значения и более того, возможно присутствует взаимосвязь в этих пропущенных данных.

Колонка `days_employed` подразумевает собой общий трудовой стаж в днях, а как видно по значениям в шапке таблицы, значения отрицательные и аномально большие, скорее всего значения взяты из 2 таблиц: в одной отрицательные, в другой положительные в часах, поэтому отрицательные надо привести к положительным, а положительные поделить на 24 часа и тогда данные будут соответствовать количеству отработанных дней. 

Значения слолбцов `days_employed`, `total_income` имеют тип float64, с числами после запятой - логично стаж работы перевести в тип int64, так как стаж целое число, а ежемесячный доход округлить до 2 знаков после запятой.

Столбец `children` есть выбросы со значением -1 в количестве детей - их приведём в положительное значение.

В столбце `dob_years` закралась ошибка со значением 0 лет, которого не может быть. Очевидно это ошибка. Проверю нет ли взаимосвязи с другими данными в таблице. Если нет, значит данные полностью случайны,  их небольшой процент от общего количества и строки с с возрастом 0 можно смело удалять, это не повлияет на результаты исследования.

Столбец `education` очевидно наличие дубликатов, от которых мы избавимся далее.

Столбец `gender` одно значение XNA, которое очевидно ошибка и строку с ним следует удалить.

Столбец `purpose` по нему много дублирований одних и тех же целей получения кредита с разной интерпритацией. Здесь воспользуемся леммитизацией и сгруппируем данные по одним целям с разным написанием.

Данных достаточно для проведения исследования.

## Шаг 2. Предобработка данных

### Обработка пропусков
   
  

In [8]:
# Столбец children значения -1 в количестве детей приведём к положительному значению
df.loc[df['children'] == -1, 'children'] = 1
print(df['children'].unique()) # проверка, что всё выполнилось чётко -1 заменилось на 1

[ 1  0  3  2  4 20  5]


In [9]:
# проверим стоблбец dob_years на предмет взаимосвязи с другими данными таблицы
df[df['dob_years'] == 0].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль


In [10]:
# так как никакой взаимосвязи нет, то удаляем с таблицы те строки, в которых возраст 0 и проверяем, 
# что всё корректно удалено
df = df.loc[df['dob_years'] != 0]
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75], dtype=int64)

In [11]:
# удалим строку со значением `XNA` в столбце gender и проверим корректность выполнения
df = df.loc[df['gender'] != 'XNA']
df['gender'].unique()

array(['F', 'M'], dtype=object)

Займёмся тем, что разберёмся с пропусками в столбцах `days_employed`, `total_income` и посмотрим, есть ли взаимосвязь в этих пропусках.

In [12]:
# выбираем пропущенные значения стажа, просматривем шапку из 10 строк таблицы и смотрим взаимосвязь 
# этих пропущенных значений с другими данными таблицы
df[df['days_employed'].isna() == True].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


   Как видно из выборки по пропущенным значениям, значения столбцов `days_employed` и `total_income` взаимосвязаны между собой. Отсутствие данных в этих столбцах взаимосвязаны. ПО остальным столбцам никакой связи нет с пропусками значений.
  
  Так как это количественные ячейки в этих столбцах, то можем попробовать заполнить эти пропуски в столбце `total_income` и `days_employed` посчитаем медиану - это будет более верная количественная характеристика, так как возможны выбросы. Значения медиан сгруппируем по семейному статусу и уровню образования, так это будет более близкое значение к конкретным данным.
  
   

In [13]:
# в столбце 'days_employed' значения с минусом сделаем положительными
df.loc[df['days_employed'] < 0, 'days_employed'] = df.loc[df['days_employed'] < 0, 'days_employed'].abs()
# в столбце 'days_employed' положительные значения поделим на 24 и приведем к дням (значения от 30 000, отрицательные до 18 000)
df.loc[df['days_employed'] > 20000, 'days_employed'] = df.loc[df['days_employed'] > 20000, 'days_employed'] / 24

In [14]:
print('Количество пропусков стажа:', df['days_employed'].isna().sum())
print('Количество пропусков ежемесячного дохода:', df['total_income'].isna().sum())
# создаём фрейм с группированными по id семейного статуса и уровня образования и подсчитанными 
# медианами по столбцам
medians = (df.groupby(['family_status_id', 'education_id'])
           .agg({'days_employed': 'median', 'total_income': 'median'})
           .rename(columns = {'days_employed': 'median_days_employed',
                              'total_income': 'median_total_income'}))
# присоединяем получившиеся столбцы с медианой к основному датафрейму 
df = df.merge(medians, on=['family_status_id', 'education_id'])
# значения с NaN меняем на медианные в стобцах общий трудовой стаж и месячный доход
df.loc[df['days_employed'].isna(), 'days_employed'] = (df.loc[df['days_employed'].isna(),
                                                              'median_days_employed'])
df.loc[df['total_income'].isna(), 'total_income'] = (df.loc[df['total_income'].isna(),
                                                            'median_total_income'])
print('Количество пропусков стажа после преобразования:', df['days_employed'].isna().sum())
print('Количество пропусков ежемесячного дохода после преобразования:', df['total_income'].isna().sum())

Количество пропусков стажа: 2164
Количество пропусков ежемесячного дохода: 2164
Количество пропусков стажа после преобразования: 0
Количество пропусков ежемесячного дохода после преобразования: 0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21423 entries, 0 to 21422
Data columns (total 14 columns):
children                21423 non-null int64
days_employed           21423 non-null float64
dob_years               21423 non-null int64
education               21423 non-null object
education_id            21423 non-null int64
family_status           21423 non-null object
family_status_id        21423 non-null int64
gender                  21423 non-null object
income_type             21423 non-null object
debt                    21423 non-null int64
total_income            21423 non-null float64
purpose                 21423 non-null object
median_days_employed    21423 non-null float64
median_total_income     21423 non-null float64
dtypes: float64(4), int64(5), object(5)
memory usage: 2.5+ MB


Мы обработали основные данные в таблицу, избавились от пропусков и аномальных значений

### Замена типа данных

In [16]:
# приводим к целом значениям стаж в днях
df['days_employed'] = df['days_employed'].astype('int')
# округляем до двух знаков месячный заработок
df['total_income'] = round(df['total_income'],ndigits=2)
# проверка выполнения замены
print(df.loc[:,['days_employed', 'total_income']])

       days_employed  total_income
0               8437     253875.64
1               2879     240525.97
2               4171     113943.49
3                529     308848.98
4                717     187863.24
...              ...           ...
21418           5352     268411.21
21419           5968     111392.23
21420          15678     255425.20
21421            409     198570.76
21422           2351     115949.04

[21423 rows x 2 columns]


**Вывод**

По типам данным, мы заменили тип стажа работы с float на int и округлили ежемесячную зарплату до копеек.

### Обработка дубликатов

In [17]:
# привели значения в столбце education к нижнему регистру
df.loc[:,'education'] = df.loc[:,'education'].str.lower()

In [18]:
print('Дубликатов всего: ', df.duplicated().sum())
# keep = False для вывода полных совпадений дубликатов
# df[df.duplicated(keep = False)].sort_values(by = list(df.columns)) # keep = False
# удалим дубликаты в нашей таблице, так как они единичны, то вызваны сбоем вероятно техническим
df = df.drop_duplicates()
print('Дубликатов после удаления: ', df.duplicated().sum())

Дубликатов всего:  71
Дубликатов после удаления:  0


**Вывод**

В таблице оказался 71 дубликат, от которых мы избавились.

### Лемматизация

In [ ]:
# создаём объект класса Mystem для лемматизации цели получения кредита
m = Mystem()
# получаем список уникальных значений столбца "purpose" в списке
list_purpose = ' '.joinin(df['purpose'].unique())
# лемматизируем список
lemmas = m.lemmatize(list_purpose)
# полуаем словарь самых употребляемых слов в списке
print(Counter(lemmas))

In [ ]:
# создаю список с уникальными леммами-существительными для лематизации каждой ячейки столбца 'purpose'
list_lemma_purpose_unique = ['недвижимость', 'автомобиль', 'образование', 'жилье', 'свадьба']
# функция, лемматизирующая строку данных, сравнивающая её со списком и возвращающее значение сравнения
def lemmas_purpose(lemma_value, list_lemma_purpose_unique=list_lemma_purpose_unique):
    lemma = m.lemmatize(lemma_value)
    for i in list_lemma_purpose_unique:
        if i in lemma:
            return i
    return 'Леммы не нашлось'
# добавляем столбец лемматизирующий столбец 'purpose'
df['lemma_purpose'] = df['purpose'].apply(lemmas_purpose)
df.head()

**Вывод**

Лемматизировали столбец 'цель получения дохода'

### Категоризация данных

In [ ]:
# создадим дополнительный столбец, который будет показывать два значения столбца 'children' 
# - есть дети или нет в зависимости от наличия
# функция, которая возвращает 'нет детей', если детей 0, и 'есть дети', если детей больше или равно 1
def family(child):
    if child == 0:
        return 'нет детей'
    else:
        return 'есть дети'
# добавляем столбец в таблицу с данными о наличии или отсутствии детей
df['child'] = df['children'].apply(family)

In [ ]:
# создадим дополнительный столбец, который будет показывать два значения столбца 'family_status' 
# - семья есть или семьи нет
# функция, которая возвращает 'семья ест', если 'женат / замужем' или 'гражданский брак', 
# и 'семья отсутствует' в иных случаях
def family_have(status):
    if (status == 'женат / замужем') or (status == 'гражданский брак'):
        return 'семья есть'
    else:
        return 'семья отсутствует'
# добавляем столбец в таблицу с данными о наличии или отсутствии семьи
df['family_have'] = df['family_status'].apply(family_have)

**Вывод**

Создал два дополнительных столбца, которые категоризуют семейное положение и наличие детей, котопые пригодятся далее для анализа

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
df_child = df.pivot_table(index=['child'], columns='debt', 
                              values='children', aggfunc='count')
df_child['%'] = df_child[1] / (df_child[1] + df_child[0]) * 100
display(df_child)

**Вывод**

Наличие детей увеличивает шансы иметь задолженность по возврату кредитов

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
# строим сводные таблицы по семейному статусу и задолженностью по кредитам (0 - не имеет задолженностей, 
# 1 - имеет задолженность по кредиту)
# сводная таблица по 'family_have','family_status'
family_debt = (df.pivot_table(index=['family_have','family_status'], columns='debt', 
                              values='family_status_id', aggfunc='count'))

# добавляем столбец с процентами  имеющими задолженности к неимеющим 
family_debt['%family_status'] = (family_debt[1] / (family_debt[1] + family_debt[0]) * 100)
# сводная таблица по 'family_have'
family_debt_family_have = (df.pivot_table(index='family_have', columns='debt', 
                              values='family_status_id', aggfunc='count'))
# добавляем столбец с процентами  имеющими задолженности к неимеющим в общем разрезе семьи
family_debt_family_have['%family_have'] = (family_debt_family_have[1] / 
                            (family_debt_family_have[1] + family_debt_family_have[0]) * 100)
# вывод сводных данных по зависимостей семейного положения и задолженностей по кредитам
display(family_debt)
display(family_debt_family_have)

**Вывод**

Видим, что наименьший процент людей, имеющих задолженность по возврату кредитов имеют `вдовец / вдова`, самые ненадежные заемщики `Не женат / не замужем`. В целом семейные заёмщики немного более надежные, чем не семейные.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
# сводная таблица считает медиану дохода по полу клиента и по наличии или отсутствии ранее задолженностей
# по возврату кредитов
df.pivot_table(index='gender', columns='debt', values='total_income', aggfunc='median')

In [ ]:
# смотрю раброс дохода, чтобы выделить в группы
print(df['total_income'].min(), df['total_income'].median(), 
      round(df['total_income'].mean(),2), df['total_income'].max())

In [ ]:
# выделю 3 группы дохода (до 100 000 - небольшой доход, от 100 000 включительно до 180 000 - средний доход
# и от 180 000 включительно высокий доход)
def total_income_have(income):
    if income < 100000:
        return 'низкий доход'
    elif income >= 180000:
        return 'высокий доход'
    else:
        return 'средний доход'
# добавляем столбец в таблицу с данными о категории дохода
df['total_income_have'] = df['total_income'].apply(total_income_have)
# проверяю разбивку по количеству данных в каждой группе
df['total_income_have'].value_counts()

In [ ]:
# сводная таблица по категориям дохода
df_total_income_have = (df.pivot_table(index='total_income_have', columns='debt', 
                                       values='total_income', aggfunc='count'))
# добавляем столбец с процентом имеющих задолженность ко всем по каждой из категорий
df_total_income_have['%'] = (df_total_income_have[1] /
                             (df_total_income_have[1] + df_total_income_have[0]) * 100)
display(df_total_income_have)

Исходя из анализа получается, что **заёмщики с высоким уровнем дохода имеют меньше проблем** с задолженностью по кредитам, а вот самые **проблемные заёмщики** имеют именно **средний доход**, а не низкий. </div>

**Вывод**

По выводу данных в сводной таблицы, если заёмщик женкого пола, то доход не имеет влияние на задолженность по кредиту. Если заёмщик мужчина, более высокий доход имеют надёжные заемщики.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
df_purpose = df.pivot_table(index='lemma_purpose', columns='debt', values='purpose', aggfunc='count')
df_purpose['%'] = df_purpose[1] / (df_purpose[1] + df_purpose[0]) * 100
display(df_purpose)

**Вывод**

Заёмщики на получение кредита на стоительство недвижимости и операции связанные с жильём самые надёжные. Кредиты на автомобили и образование имеют больший процент задолженностей по возврату кредита.

## Шаг 4. Общий вывод

**По основному вопросу заказчика: влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок можно сделать вывод, что наличие детей может усложнить погашение кредита и имеет негативное влияние на кредитную историю, а семейный человек в целом немного более надёжный заёмщик, чем  не семейный.
Самый надежный заемщик вдовец/вдова или в разводе без наличия детей.**